In [1]:
!pip install bokeh --quiet

In [2]:
!pip show bokeh

Name: bokeh
Version: 3.2.0
Summary: Interactive plots and applications in the browser from Python
Home-page: 
Author: Bokeh Team
Author-email: info@bokeh.org
License: Copyright (c) 2012 - 2023, Anaconda, Inc., and Bokeh Contributors
        All rights reserved.
        
        Redistribution and use in source and binary forms, with or without modification,
        are permitted provided that the following conditions are met:
        
        Redistributions of source code must retain the above copyright notice,
        this list of conditions and the following disclaimer.
        
        Redistributions in binary form must reproduce the above copyright notice,
        this list of conditions and the following disclaimer in the documentation
        and/or other materials provided with the distribution.
        
        Neither the name of Anaconda nor the names of any contributors
        may be used to endorse or promote products derived from this software
        without specific p

In [1]:
conda activate heatmap


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [13]:
import sys
print(sys.executable)

/opt/conda/envs/heatmap/bin/python3.9


### This code should work, but there is not the correct import for the countries, so trying a new thing next code block

In [3]:
from datetime import datetime
from bokeh.plotting import figure, show, output_file
from bokeh.models import GeoJSONDataSource, LogColorMapper
from bokeh.palettes import Viridis256
from bokeh.sampledata import countries
import pandas as pd

# Load the dataset
data = pd.read_csv('owid-covid-data.csv')

# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'])

# Specify the desired day to isolate
target_date = datetime(2023, 6, 28)

# Filter the data for the desired day
filtered_data = data[data['date'] == target_date]

# Create a GeoJSONDataSource from the country boundaries
geosource = GeoJSONDataSource(geojson=countries.data)

# Create a color mapper for the choropleth
color_mapper = LogColorMapper(palette=Viridis256)

# Create a Bokeh figure
p = figure(title='Choropleth Heatmap', plot_width=800, plot_height=500)

# Add country boundaries and color them based on occurrence
p.patches('xs', 'ys', source=geosource, fill_color={'field': 'new_cases_smoothed', 'transform': color_mapper},
          line_color='gray', line_width=0.5, fill_alpha=1)

# Add a color bar
color_bar = figure(title='New Cases Smoothed', title_location='above', toolbar_location='below',
                   plot_width=250, plot_height=20)
color_bar.toolbar.logo = None
color_bar.toolbar_location = None
color_bar.min_border_top = 10
color_bar.min_border_bottom = 0
color_bar.min_border_left = 10
color_bar.min_border_right = 10
color_bar.rect(x=0, y=0, width=1, height=1, fill_color={'field': 'new_cases_smoothed', 'transform': color_mapper},
               line_color=None)
p.add_layout(color_bar, 'below')

# Output the heatmap to an HTML file
output_file('testheatmap.html')

# Show the Bokeh figure
show(p)


ImportError: cannot import name 'countries' from 'bokeh.sampledata' (/opt/conda/envs/heatmap/lib/python3.9/site-packages/bokeh/sampledata/__init__.py)

In [ ]:
from datetime import datetime
from bokeh.plotting import figure, show, output_file
from bokeh.models import GeoJSONDataSource, LogColorMapper
from bokeh.palettes import Viridis256
import pandas as pd
import json

# Load the dataset
data = pd.read_csv('owid-covid-data.csv')

# Specify the desired date to create the heatmap
desired_date = '2022-01-01'

# Filter the dataset for the desired date
filtered_data = data[data['date'] == desired_date]

# Read the GeoJSON file for countries
with open('countries.geojson') as f:
    countries = json.load(f)

# Merge the filtered data with the country data
merged_data = countries.copy()
for feature in merged_data['features']:
    country_name = feature['properties']['name']
    if country_name in filtered_data['location'].values:
        occurrences = filtered_data[filtered_data['location'] == country_name]['new_cases_smoothed'].values[0]
        feature['properties']['occurrences'] = occurrences
    else:
        feature['properties']['occurrences'] = 0

# Create a GeoJSONDataSource
geo_source = GeoJSONDataSource(geojson=json.dumps(merged_data))

# Create the color mapper
color_mapper = LogColorMapper(palette=Viridis256)

# Create the figure
p = figure(title='Heatmap', width=800, height=500, toolbar_location='above', tools='pan,wheel_zoom,reset')


# Add the country polygons with color mapping
p.patches('xs', 'ys', fill_alpha=0.7, line_color='black', line_width=0.5,
          fill_color={'field': 'occurrences', 'transform': color_mapper},
          source=geo_source)

# Specify the output file
output_file('heatmap.html')
